In [1]:
import pathlib
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.ndimage
import scipy.signal
import skimage
import skimage.io as io
import skimage.morphology
import skimage.segmentation
from centrosome.cpmorphology import fixup_scipy_ndimage_result as fix
from scipy.linalg import lstsq

sys.path.append("../featurization")
import numpy
from area_size_shape import calulate_surface_area, measure_3D_area_size_shape
from colocalization import calculate_3D_colocalization
from granularity import measure_3D_granularity
from intensity import measure_3D_intensity
from loading_classes import Featurization, ImageSetLoader, ObjectLoader
from neighbors import measure_3D_number_of_neighbors
from texture import measure_3D_texture

In [2]:
channel_mapping = {
    "nuclei": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "nuclei_mask": "nuclei_",
    "cell_mask": "cell_",
    "cytoplasm_mask": "cytoplasm_",
    "organoid_mask": "organoid_",
}

In [3]:
image_set_path = pathlib.Path("../../data/NF0014/cellprofiler/C4-2/")

In [4]:
files = sorted(image_set_path.glob("*"))
files = [f for f in files if f.suffix in [".tif", ".tiff"]]

# Load images into a dictionary
image_set_dict = {}
for f in files:
    for key, value in channel_mapping.items():
        if value in f.name:
            image_set_dict[key] = io.imread(f)

In [5]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    spacing=(1, 0.1, 0.1),
    channel_mapping=channel_mapping,
)
image_set_loader.retrieve_image_attributes()
image_set_loader.unique_objects.keys()
image_set_loader.unique_objects

{'cell_mask': array([  0,   2,   4,   7,   9,  12,  14,  17,  19,  22,  24,  26,  29,
         31,  34,  36,  39,  41,  44,  46,  49,  51,  53,  56,  58,  61,
         63,  66,  68,  71,  73,  76,  78,  80,  83,  85,  88,  90,  93,
         95,  98, 100, 102, 105, 107, 110, 112, 115, 117, 120, 122, 125,
        127, 129, 132, 134, 137, 139, 142, 144, 147, 149, 152, 154, 156,
        159, 161, 164, 166, 169, 171, 174, 176, 178, 181, 183, 186, 188,
        191, 193, 196, 198, 201, 203, 205, 208, 210, 213, 215, 218, 220,
        223, 225, 228, 230, 232, 235, 237, 240, 242, 245, 247, 250, 252,
        255], dtype=uint8),
 'cytoplasm_mask': array([  0,   2,   4,   7,   9,  12,  14,  17,  19,  22,  24,  26,  29,
         31,  34,  36,  39,  41,  44,  46,  49,  51,  53,  56,  58,  61,
         63,  66,  68,  71,  73,  76,  78,  80,  83,  85,  88,  90,  93,
         95,  98, 100, 102, 105, 107, 110, 112, 115, 117, 120, 122, 125,
        127, 129, 132, 134, 137, 139, 142, 144, 147, 149, 152, 15

In [ ]:
# iterate through each unique object in each compartment for each channel image

for channel_image in image_set_loader.get_image_names():
    for compartment in image_set_loader.get_compartments():
        for unique_object_index in image_set_loader.unique_objects[compartment]:
            print(
                f"Channel: {channel_image}, Compartment: {compartment}, Object: {unique_object_index}"
            )
            # initialize object loader for each unique object
            objects_loader = ObjectLoader(
                image=image_set_loader.get_image(channel_image),
                label_image=image_set_loader.get_image(compartment),
                channel_name=channel_image,
                compartment_name=compartment,
                label_index=unique_object_index,
            )
            # featurize the object

In [6]:
objects_loader = ObjectLoader(
    image=image_set_loader.get_image("nuclei"),
    label_image=image_set_loader.get_image("nuclei_mask"),
    channel_name="nuclei",
    compartment_name="nuclei",
    label_index=4,
)

In [7]:
featurization = Featurization(
    image_set_loader=image_set_loader,
    object_loader=objects_loader,
    neighbors_distance_threshold=10,
)
features = featurization.get_features()

INFO:root:Calculated Area Size Shape features for nuclei 4
INFO:root:Calculated Granularity features for nuclei 4
INFO:root:Calculated Neighbors features for nuclei 4
INFO:root:Calculated Adjacent Neighbors features for nuclei 4


In [8]:
features

{'area_size_shape': {'VOLUME': array([88442.]),
  'SURFACE_AREA': 597.6736509775509,
  'MAJOR_AXIS_LENGTH': array([156.89838983]),
  'MINOR_AXIS_LENGTH': array([12.31554867]),
  'CENTER_X': array([742.10532326]),
  'CENTER_Y': array([386.92353181]),
  'CENTER_Z': array([5.31414939]),
  'BBOX_VOLUME': array([131130.]),
  'MIN_X': array([671]),
  'MAX_X': array([812]),
  'MIN_Y': array([343]),
  'MAX_Y': array([436]),
  'MIN_Z': array([1]),
  'MAX_Z': array([11]),
  'EXTENT': array([0.67446046]),
  'EULER_NUMBER': array([1]),
  'EQUIVALENT_DIAMETER': array([55.27812593])},
 'granularity': {'Granularity_1_nuclei': 100.0,
  'Granularity_2_nuclei': 0.6213558752982083,
  'Granularity_3_nuclei': 0.003860831237676026,
  'Granularity_4_nuclei': 2.398950173064852e-05,
  'Granularity_5_nuclei': 1.4906017845814993e-07},
 'intensity': {'integrated_intensity': 1452409130,
  'volume': 353768,
  'mean_intensity': 4105.541286945117,
  'std_intensity': 4441.863521884266,
  'min_intensity': 6508,
  'max_

In [10]:
compartment = "Nuclei"
channel = "DNA"
label_index = 4

In [11]:
updated_features = {}
for key, value in features.items():
    if isinstance(value, dict):
        for sub_key, sub_value in value.items():
            updated_features[
                f"{compartment}_{channel}_{key}_{sub_key}_{label_index}"
            ] = sub_value
    else:
        updated_features[
            f"{compartment}_{channel}_{key}_nosubfeature_{label_index}"
        ] = value
updated_features

{'Nuclei_DNA_area_size_shape_VOLUME_4': array([88442.]),
 'Nuclei_DNA_area_size_shape_SURFACE_AREA_4': 597.6736509775509,
 'Nuclei_DNA_area_size_shape_MAJOR_AXIS_LENGTH_4': array([156.89838983]),
 'Nuclei_DNA_area_size_shape_MINOR_AXIS_LENGTH_4': array([12.31554867]),
 'Nuclei_DNA_area_size_shape_CENTER_X_4': array([742.10532326]),
 'Nuclei_DNA_area_size_shape_CENTER_Y_4': array([386.92353181]),
 'Nuclei_DNA_area_size_shape_CENTER_Z_4': array([5.31414939]),
 'Nuclei_DNA_area_size_shape_BBOX_VOLUME_4': array([131130.]),
 'Nuclei_DNA_area_size_shape_MIN_X_4': array([671]),
 'Nuclei_DNA_area_size_shape_MAX_X_4': array([812]),
 'Nuclei_DNA_area_size_shape_MIN_Y_4': array([343]),
 'Nuclei_DNA_area_size_shape_MAX_Y_4': array([436]),
 'Nuclei_DNA_area_size_shape_MIN_Z_4': array([1]),
 'Nuclei_DNA_area_size_shape_MAX_Z_4': array([11]),
 'Nuclei_DNA_area_size_shape_EXTENT_4': array([0.67446046]),
 'Nuclei_DNA_area_size_shape_EULER_NUMBER_4': array([1]),
 'Nuclei_DNA_area_size_shape_EQUIVALENT_D